In [88]:
from datasets import load_dataset

ds = load_dataset("Grandediw/clash-royale-battle")

# Convert the dataset to a Pandas DataFrame
df_battle = ds['train'].to_pandas()

df_battle = df_battle.head(1000)

# Display the first few rows
print("Converted Dataset to DataFrame:")
print(df_battle.head())

Converted Dataset to DataFrame:
   Unnamed: 0                 battleTime    arena.id  gameMode.id  \
0           0  2020-12-31 21:02:12+00:00  54000050.0   72000006.0   
1           1  2020-12-31 21:02:15+00:00  54000050.0   72000006.0   
2           2  2020-12-31 21:02:45+00:00  54000050.0   72000006.0   
3           3  2020-12-31 21:03:13+00:00  54000050.0   72000006.0   
4           4  2020-12-31 21:03:17+00:00  54000050.0   72000006.0   

   average.startingTrophies  winner.tag  winner.startingTrophies  \
0                    5363.0   #PVLPJP2Y                   5372.0   
1                    5407.0  #8PRLRYYCV                   5409.0   
2                    5741.0   #2G8LQRCG                   5749.0   
3                    4307.0  #Y9QL09VGV                   4316.0   
4                    5776.5  #9RRYG9P9U                   5783.0   

   winner.trophyChange  winner.crowns  winner.kingTowerHitPoints  \
0                 28.0            2.0                     4145.0   
1       

In [89]:
import ast
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Use only the first 100,000 rows
df_battle = df_battle.head(100000)

# Define the card mapping dictionary
card_mapping = {
    26000000: "Knight", 26000001: "Archers", 26000002: "Goblins", 26000003: "Giant", 26000004: "P.E.K.K.A",
    26000005: "Minions", 26000006: "Balloon", 26000007: "Witch", 26000008: "Barbarians", 26000009: "Golem",
    26000010: "Skeletons", 26000011: "Valkyrie", 26000012: "Skeleton Army", 26000013: "Bomber",
    26000014: "Musketeer", 26000015: "Baby Dragon", 26000016: "Prince", 26000017: "Wizard",
    26000018: "Mini P.E.K.K.A", 26000019: "Spear Goblins", 26000020: "Giant Skeleton", 26000021: "Hog Rider",
    26000022: "Minion Horde", 26000023: "Ice Wizard", 26000024: "Royal Giant", 26000025: "Guards",
    26000026: "Princess", 26000027: "Dark Prince", 26000028: "Three Musketeers", 26000029: "Lava Hound",
    26000030: "Ice Spirit", 26000031: "Fire Spirits", 26000032: "Miner", 26000033: "Sparky",
    26000034: "Bowler", 26000035: "Lumberjack", 26000036: "Battle Ram", 26000037: "Inferno Dragon",
    26000038: "Ice Golem", 26000039: "Mega Minion", 26000040: "Dart Goblin", 26000041: "Goblin Gang",
    26000042: "Electro Wizard", 26000043: "Elite Barbarians", 26000044: "Hunter", 26000045: "Executioner",
    26000046: "Bandit", 26000047: "Royal Recruits", 26000048: "Night Witch", 26000049: "Bats",
    26000050: "Royal Ghost", 26000051: "Ram Rider", 26000052: "Zappies", 26000053: "Rascals",
    26000054: "Cannon Cart", 26000055: "Mega Knight", 26000056: "Skeleton Barrel", 26000057: "Flying Machine",
    26000058: "Wall Breakers", 26000059: "Royal Hogs", 26000060: "Goblin Giant", 26000061: "Fisherman",
    26000062: "Magic Archer", 26000063: "Electro Dragon", 26000064: "Firecracker", 26000067: "Elixir Golem",
    26000068: "Battle Healer", 26000080: "Skeleton Dragons", 26000083: "Mother Witch", 26000084: "Electro Spirit",
    26000085: "Electro Giant", 27000000: "Cannon", 27000001: "Goblin Hut", 27000002: "Mortar",
    27000003: "Inferno Tower", 27000004: "Bomb Tower", 27000005: "Barbarian Hut", 27000006: "Tesla",
    27000007: "Elixir Collector", 27000008: "X-Bow", 27000009: "Tombstone", 27000010: "Furnace",
    27000012: "Goblin Cage", 28000000: "Fireball", 28000001: "Arrows", 28000002: "Rage",
    28000003: "Rocket", 28000004: "Goblin Barrel", 28000005: "Freeze", 28000006: "Mirror",
    28000007: "Lightning", 28000008: "Zap", 28000009: "Poison", 28000010: "Graveyard",
    28000011: "The Log", 28000012: "Tornado", 28000013: "Clone", 28000014: "Earthquake",
    28000015: "Barbarian Barrel", 28000016: "Heal Spirit", 28000017: "Giant Snowball", 28000018: "Royal Delivery"
}

# Process the cards list
df_battle.loc[:, 'loser.cards.list'] = df_battle['loser.cards.list'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
df_battle.loc[:, 'winner.cards.list'] = df_battle['winner.cards.list'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Convert card IDs to names
df_battle.loc[:, 'loser.cards.list.names'] = df_battle['loser.cards.list'].apply(
    lambda card_ids: [card_mapping.get(card_id, "Unknown") for card_id in card_ids]
)
df_battle.loc[:, 'winner.cards.list.names'] = df_battle['winner.cards.list'].apply(
    lambda card_ids: [card_mapping.get(card_id, "Unknown") for card_id in card_ids]
)

# Combine all unique cards
winner_cards = df_battle['winner.cards.list.names'].explode()
loser_cards = df_battle['loser.cards.list.names'].explode()
all_cards = pd.concat([winner_cards, loser_cards]).unique()
all_cards = sorted(all_cards)

# Evaluate winners and one-hot encode decks
player_results = []
for _, row in df_battle.iterrows():
    player_results.append({
        "player_name": row["winner.tag"],
        "deck": row["winner.cards.list.names"],
        "result": 1,  # Change "WIN" to 1
    })
    if row["loser.cards.list.names"]:  # Check for valid loser data
        player_results.append({
            "player_name": row.get("loser.tag"),
            "deck": row["loser.cards.list.names"],
            "result": 0,  # Change "LOSE" to 0
        })

# Create final DataFrame
result_df = pd.DataFrame(player_results)

# Ensure "Result" is the last column
result_df = result_df[["player_name", "deck", "result"]]

# Display the processed DataFrame
print("Processed DataFrame with Numeric Results:")
print(result_df.head())


Processed DataFrame with Numeric Results:
  player_name  \
0   #PVLPJP2Y   
1  #PLYJVUQY2   
2  #8PRLRYYCV   
3   #92VG2CPY   
4   #2G8LQRCG   

                                                                                                 deck  \
0                     [Giant, P.E.K.K.A, Balloon, Barbarians, Minion Horde, Bomb Tower, Arrows, Rage]   
1         [Knight, Baby Dragon, Ice Wizard, Bomb Tower, Poison, Graveyard, Tornado, Barbarian Barrel]   
2  [Knight, Skeletons, Princess, Dart Goblin, Skeleton Barrel, Inferno Tower, Goblin Barrel, The Log]   
3    [Ice Wizard, Dark Prince, Inferno Dragon, Bandit, Mega Knight, Magic Archer, Goblin Hut, Poison]   
4      [Skeletons, Electro Wizard, Hunter, Mega Knight, Royal Hogs, Fisherman, Zap, Barbarian Barrel]   

   result  
0       1  
1       0  
2       1  
3       0  
4       1  


In [90]:
# Dictionary mapping card names to IDs
card_numbers = {
    "Archers": 1, "Archer Queen": 2, "Baby Dragon": 3, "Balloon": 4, "Bandit": 5, "Barbarians": 6,
    "Bats": 7, "Battle Healer": 8, "Battle Ram": 9, "Bomber": 10, "Bowler": 11, "Bush Goblins": 12,
    "Cannon Cart": 13, "Cursed Hog": 14, "Dark Prince": 15, "Dart Goblin": 16, "Electro Dragon": 17,
    "Electro Giant": 18, "Electro Spirit": 19, "Electro Wizard": 20, "Elite Barbarians": 21,
    "Elixir Blob": 22, "Elixir Golem": 23, "Elixir Golemite": 24, "Executioner": 25, "Firecracker": 26,
    "Fire Spirit": 27, "Fisherman": 28, "Flying Machine": 29, "Giant": 30, "Giant Skeleton": 31,
    "Goblin Brawler": 32, "Goblin Gang": 33, "Goblin Demolisher": 34, "Goblin Giant": 35,
    "Goblin Machine": 36, "Goblins": 37, "Goblinstein": 38, "Golden Knight": 39, "Golem": 40,
    "Golemite": 41, "Guardienne": 42, "Guards": 43, "Hog Rider": 44, "Hunter": 45, "Heal Spirit": 46,
    "Ice Golem": 47, "Ice Spirit": 48, "Ice Wizard": 49, "Inferno Dragon": 50, "Knight": 51,
    "Lava Hound": 52, "Lava Pup": 53, "Little Prince": 54, "Lumberjack": 55, "Magic Archer": 56,
    "Mega Knight": 57, "Mega Minion": 58, "Mighty Miner": 59, "Miner": 60, "Mini P.E.K.K.A.": 61,
    "Minion Horde": 62, "Minions": 63, "Monk": 64, "Mother Witch": 65, "Monster": 66, "Musketeer": 67,
    "Night Witch": 68, "P.E.K.K.A.": 69, "Phoenix": 70, "Reborn Phoenix": 71, "Prince": 72,
    "Princess": 73, "Ram Rider": 74, "Rascal Boy": 75, "Rascal Girl": 76, "Royal Ghost": 77,
    "Royal Giant": 78, "Royal Hogs": 79, "Royal Recruits": 80, "Skeleton Army": 81,
    "Skeleton Barrel": 82, "Skeleton Dragons": 83, "Skeleton King": 84, "Skeletons": 85, "Sparky": 86,
    "Spear Goblins": 87, "Suspicious Bush": 88, "Three Musketeers": 89, "Valkyrie": 90,
    "Wall Breakers": 91, "Witch": 92, "Wizard": 93, "Zappies": 94,"Bomb Tower": 95, "Cannon": 96, "Cannon Cart (broken)": 97, "Inferno Tower": 98, "Mortar": 99,
        "Tesla": 100, "X-Bow": 101,"Barbarian Hut": 102, "Elixir Collector": 103, "Furnace": 104, "Goblin Cage": 105,
        "Goblin Drill": 106, "Goblin Hut": 107, "Phoenix Egg": 108, "Tombstone": 109, "Arrows": 110, "Barbarian Barrel": 111, "Earthquake": 112, "Fireball": 113, "Freeze": 114,
        "Giant Snowball": 115, "Goblin Curse": 116, "Lightning": 117, "Poison": 118, "Rage": 119, "Rocket": 120, "Royal Delivery": 121, "The Log": 122, "Tornado": 123, "Void": 124, "Zap": 125,
        "Barbarian Barrel": 126, "Barbarian Hut": 127, "Battle Ram": 128, "Elixir Golem": 129,
        "Elixir Golemite": 130, "Furnace": 131, "Goblin Barrel": 132, "Goblin Cage": 133, "Goblin Curse": 134,
        "Goblin Drill": 135, "Goblin Giant": 136, "Goblin Hut": 137, "Golem": 138, "Graveyard": 139,
        "Lava Hound": 140, "Little Prince": 141, "Mother Witch": 142, "Night Witch": 143,
        "Phoenix Egg": 144, "Royal Delivery": 145, "Skeleton Barrel": 146, "Skeleton King": 147,
        "Suspicious Bush": 148, "Tombstone": 149, "Witch": 150,
        "Archers/Evolution": 155, "Barbarians/Evolution": 156, "Battle Ram/Evolution": 157,
        "Bats/Evolution": 158, "Bomber/Evolution": 159, "Cannon/Evolution": 160,
        "Electro Dragon/Evolution": 161, "Firecracker/Evolution": 162, "Giant Snowball/Evolution": 163,
        "Goblin Barrel/Evolution": 164, "Goblin Cage/Evolution": 165, "Goblin Drill/Evolution": 166,
        "Goblin Giant/Evolution": 167, "Ice Spirit/Evolution": 168, "Knight/Evolution": 169,
        "Mega Knight/Evolution": 170, "Mortar/Evolution": 171, "Musketeer/Evolution": 172,
        "P.E.K.K.A/Evolution": 173, "Royal Giant/Evolution": 174, "Royal Recruits/Evolution": 175,
        "Skeletons/Evolution": 176, "Tesla/Evolution": 177, "Valkyrie/Evolution": 178,
        "Wall Breakers/Evolution": 179, "Wizard/Evolution": 180, "Zap/Evolution": 181
}

# Ensure card names are mapped to numbers
def deck_to_ids(deck, mapping):
    return [mapping.get(card, 0) - 1 for card in deck]  # Subtract 1 for zero-based indexing

result_df["Deck IDs"] = result_df["deck"].apply(lambda deck: deck_to_ids(deck, card_numbers))

# One-hot encode the Deck IDs
num_choices = 181  # Total possible card IDs
encoder = OneHotEncoder(categories=[range(num_choices)], sparse_output=False)

# Reshape the Deck IDs column to 2D before passing it to the encoder
deck_ids_2d = result_df["Deck IDs"].apply(pd.Series).fillna(-1).astype(int)

# Transform each deck into a one-hot encoded representation
one_hot_encoded = []
for _, row in deck_ids_2d.iterrows():
    one_hot = np.zeros(num_choices, dtype=int)
    one_hot[row[row != -1]] = 1  # Set indices corresponding to cards to 1
    one_hot_encoded.append(one_hot)

result_df["deck"] = one_hot_encoded

# Drop intermediate "Deck IDs" column
result_df = result_df.drop(columns=["Deck IDs"])

# Ensure "Result" is the last column
result_df = result_df[["player_name", "deck", "result"]]

# Display the processed DataFrame
print("Processed DataFrame with One-Hot Encoded Decks:")
print(result_df)

Processed DataFrame with One-Hot Encoded Decks:
     player_name  \
0      #PVLPJP2Y   
1     #PLYJVUQY2   
2     #8PRLRYYCV   
3      #92VG2CPY   
4      #2G8LQRCG   
...          ...   
1995   #JC98JLVP   
1996   #PP82G80Y   
1997    #UQURJ20   
1998   #2RQGQ208   
1999  #92YYP9C9C   

                                                                                                                                                                                                                                                                                                                   deck  \
0     [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]   
1     [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [91]:
import requests
import json
import os
import urllib.parse


def clean_battle_data(battle_data):
    # Define keys to retain
    desired_keys = {'name', 'level', 'elixirCost', 'rarity'}

    def filter_card_data(card):
        # Filter each card's data to retain only desired keys
        return {key: card[key] for key in card if key in desired_keys}

    def process_team_or_opponent(team_or_opponent):
        # Process each team or opponent's cards
        team_or_opponent['cards'] = [filter_card_data(card) for card in team_or_opponent['cards']]
        return team_or_opponent

    # Process the entire battle data
    for battle in battle_data:
        battle['team'] = [process_team_or_opponent(member) for member in battle['team']]
        battle['opponent'] = [process_team_or_opponent(member) for member in battle['opponent']]

    return battle_data

# Example usage with your JSON data
import json

# Assuming `battle_data` is your original JSON response loaded as a Python object
#cleaned_data = clean_battle_data(battle_data)

# Pretty-print the cleaned data
#print(json.dumps(cleaned_data, indent=2))

def get_player_data(player_tag, api_key):
    # URL-encode the player tag
    #encoded_tag = urllib.parse.quote(player_tag)

    # Define the API endpoint
    url = f'https://api.clashroyale.com/v1/players/{player_tag}/battlelog'

    # Set up the headers with the API key
    headers = {
        'Accept': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    # Make the GET request to the API
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error: {response.status_code}')
        return None

if __name__ == '__main__':
    # 1. Read the API key from the 'data/clash-royale-api-key.txt' file
    key_file_path = os.path.join('../data/clash-royale-api-key.txt')
    with open(key_file_path, 'r') as file:
        api_key = file.read().strip()

    # 2. Specify the player tag (URL-encoded # is often %23 in Clash Royale)
    player_tag = '%232LGY9G'  # Replace with the actual player tag, e.g. #2LGY9G -> %232LGY9G

    # 3. Call the API
    player_data = get_player_data(player_tag, api_key)
    if player_data:
        # 4. Clean the data
        cleaned_data = clean_battle_data(player_data)

        # 5. Save cleaned data to 'outputCR.json' in the 'data' folder
        output_path = os.path.join('../data/outputCR.json')
        with open(output_path, 'w', encoding='utf-16') as out_file:
            json.dump(cleaned_data, out_file, indent=2)

        # 6. Print the JSON file contents
        print(f"Saved cleaned data to: {output_path}\n")
        print("Printing contents of outputCR.json:\n")
        with open(output_path, 'r', encoding='utf-16') as in_file:
            file_data = json.load(in_file)
            print(json.dumps(file_data, indent=2))
    else:
        print("No data received from API. Please check your player tag or API key.")

Saved cleaned data to: ../data/outputCR.json

Printing contents of outputCR.json:

[
  {
    "type": "pathOfLegend",
    "battleTime": "20250105T121523.000Z",
    "isLadderTournament": false,
    "arena": {
      "id": 54000098,
      "name": "Legendary Arena"
    },
    "gameMode": {
      "id": 72000464,
      "name": "Ranked1v1_NewArena2"
    },
    "deckSelection": "collection",
    "team": [
      {
        "tag": "#2LGY9G",
        "name": "Diwel26",
        "crowns": 1,
        "kingTowerHitPoints": 3657,
        "princessTowersHitPoints": null,
        "clan": {
          "tag": "#YULCL0YQ",
          "name": "ITA",
          "badgeId": 16000081
        },
        "cards": [
          {
            "name": "Royal Giant",
            "level": 13,
            "rarity": "common",
            "elixirCost": 6
          },
          {
            "name": "Firecracker",
            "level": 13,
            "rarity": "common",
            "elixirCost": 3
          },
          {
      

In [92]:
import pandas as pd
import json
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Dictionary mapping card names to IDs
card_numbers = {
    "Archers": 1, "Archer Queen": 2, "Baby Dragon": 3, "Balloon": 4, "Bandit": 5, "Barbarians": 6,
    "Bats": 7, "Battle Healer": 8, "Battle Ram": 9, "Bomber": 10, "Bowler": 11, "Bush Goblins": 12,
    "Cannon Cart": 13, "Cursed Hog": 14, "Dark Prince": 15, "Dart Goblin": 16, "Electro Dragon": 17,
    "Electro Giant": 18, "Electro Spirit": 19, "Electro Wizard": 20, "Elite Barbarians": 21,
    "Elixir Blob": 22, "Elixir Golem": 23, "Elixir Golemite": 24, "Executioner": 25, "Firecracker": 26,
    "Fire Spirit": 27, "Fisherman": 28, "Flying Machine": 29, "Giant": 30, "Giant Skeleton": 31,
    "Goblin Brawler": 32, "Goblin Gang": 33, "Goblin Demolisher": 34, "Goblin Giant": 35,
    "Goblin Machine": 36, "Goblins": 37, "Goblinstein": 38, "Golden Knight": 39, "Golem": 40,
    "Golemite": 41, "Guardienne": 42, "Guards": 43, "Hog Rider": 44, "Hunter": 45, "Heal Spirit": 46,
    "Ice Golem": 47, "Ice Spirit": 48, "Ice Wizard": 49, "Inferno Dragon": 50, "Knight": 51,
    "Lava Hound": 52, "Lava Pup": 53, "Little Prince": 54, "Lumberjack": 55, "Magic Archer": 56,
    "Mega Knight": 57, "Mega Minion": 58, "Mighty Miner": 59, "Miner": 60, "Mini P.E.K.K.A.": 61,
    "Minion Horde": 62, "Minions": 63, "Monk": 64, "Mother Witch": 65, "Monster": 66, "Musketeer": 67,
    "Night Witch": 68, "P.E.K.K.A.": 69, "Phoenix": 70, "Reborn Phoenix": 71, "Prince": 72,
    "Princess": 73, "Ram Rider": 74, "Rascal Boy": 75, "Rascal Girl": 76, "Royal Ghost": 77,
    "Royal Giant": 78, "Royal Hogs": 79, "Royal Recruits": 80, "Skeleton Army": 81,
    "Skeleton Barrel": 82, "Skeleton Dragons": 83, "Skeleton King": 84, "Skeletons": 85, "Sparky": 86,
    "Spear Goblins": 87, "Suspicious Bush": 88, "Three Musketeers": 89, "Valkyrie": 90,
    "Wall Breakers": 91, "Witch": 92, "Wizard": 93, "Zappies": 94,"Bomb Tower": 95, "Cannon": 96, "Cannon Cart (broken)": 97, "Inferno Tower": 98, "Mortar": 99,
        "Tesla": 100, "X-Bow": 101,"Barbarian Hut": 102, "Elixir Collector": 103, "Furnace": 104, "Goblin Cage": 105,
        "Goblin Drill": 106, "Goblin Hut": 107, "Phoenix Egg": 108, "Tombstone": 109, "Arrows": 110, "Barbarian Barrel": 111, "Earthquake": 112, "Fireball": 113, "Freeze": 114,
        "Giant Snowball": 115, "Goblin Curse": 116, "Lightning": 117, "Poison": 118, "Rage": 119, "Rocket": 120, "Royal Delivery": 121, "The Log": 122, "Tornado": 123, "Void": 124, "Zap": 125,
        "Barbarian Barrel": 126, "Barbarian Hut": 127, "Battle Ram": 128, "Elixir Golem": 129,
        "Elixir Golemite": 130, "Furnace": 131, "Goblin Barrel": 132, "Goblin Cage": 133, "Goblin Curse": 134,
        "Goblin Drill": 135, "Goblin Giant": 136, "Goblin Hut": 137, "Golem": 138, "Graveyard": 139,
        "Lava Hound": 140, "Little Prince": 141, "Mother Witch": 142, "Night Witch": 143,
        "Phoenix Egg": 144, "Royal Delivery": 145, "Skeleton Barrel": 146, "Skeleton King": 147,
        "Suspicious Bush": 148, "Tombstone": 149, "Witch": 150,
        "Archers/Evolution": 155, "Barbarians/Evolution": 156, "Battle Ram/Evolution": 157,
        "Bats/Evolution": 158, "Bomber/Evolution": 159, "Cannon/Evolution": 160,
        "Electro Dragon/Evolution": 161, "Firecracker/Evolution": 162, "Giant Snowball/Evolution": 163,
        "Goblin Barrel/Evolution": 164, "Goblin Cage/Evolution": 165, "Goblin Drill/Evolution": 166,
        "Goblin Giant/Evolution": 167, "Ice Spirit/Evolution": 168, "Knight/Evolution": 169,
        "Mega Knight/Evolution": 170, "Mortar/Evolution": 171, "Musketeer/Evolution": 172,
        "P.E.K.K.A/Evolution": 173, "Royal Giant/Evolution": 174, "Royal Recruits/Evolution": 175,
        "Skeletons/Evolution": 176, "Tesla/Evolution": 177, "Valkyrie/Evolution": 178,
        "Wall Breakers/Evolution": 179, "Wizard/Evolution": 180, "Zap/Evolution": 181

}


# One-hot encoder setup
num_choices = 181  # Total possible card IDs # 116 actual cards
encoder = OneHotEncoder(categories=[range(num_choices)], sparse_output=False)

# Assuming the provided JSON is saved in 'outputCR.json'
try:
    with open('../data/outputCR.json', 'r', encoding='utf-16') as f:
        battle_data = json.load(f)  # Load the JSON data
except FileNotFoundError:
    print("File not found. Ensure 'outputCR.json' exists in the directory.")
    battle_data = None
except json.JSONDecodeError as e:
    print(f"Invalid JSON format: {e}")
    battle_data = None

if battle_data:
    # Function to process a single player's data
    def process_player_data(player, is_team=True):
        # Convert card names to IDs and handle missing cases
        deck = [card_numbers.get(card['name'], -1) for card in player['cards']]
        # Filter out invalid card IDs (-1)
        deck = [card for card in deck if card >= 0]

        # One-hot encode the deck
        if len(deck) > 0:
            deck_one_hot = encoder.fit_transform(np.array(deck).reshape(-1, 1)).sum(axis=0)
        else:
            deck_one_hot = np.zeros(num_choices)

        trophy_change = player.get('trophyChange', 0) * (1 if is_team else -1)
        return {
            'Player Name': player['name'],
            'Deck': list(deck_one_hot), # (One-Hot Encoded)
            'Result': '',
        }

    # Collect all matches
    all_matches = []

    for match in battle_data:
        team = match['team'][0]
        opponent = match['opponent'][0]
        team_data = process_player_data(team, is_team=True)
        opponent_data = process_player_data(opponent, is_team=False)

        # Add Win or Lose based on crowns
        if team['crowns'] > opponent['crowns']:
            team_data['Result'] = 1 #'WIN'
            opponent_data['Result'] = 0 #'LOSE'
        elif team['crowns'] < opponent['crowns']:
            team_data['Result'] = 0 #'LOSE'
            opponent_data['Result'] = 1 #'WIN'
        else:
            team_data['Result'] = 2#'DRAW'
            opponent_data['Result'] =2 #'DRAW'

        all_matches.append(team_data)
        all_matches.append(opponent_data)

    # Create the DataFrame
    df = pd.DataFrame(all_matches)

    # Display the DataFrame
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)
    print(df)
else:
    print("No battle data available.")


        Player Name  \
0           Diwel26   
1       حسین شهبازی   
2           Diwel26   
3       killergamer   
4           Diwel26   
5             J.MR✓   
6           Diwel26   
7             원주민준우   
8           Diwel26   
9               갓대현   
10          Diwel26   
11    papillon d'or   
12          Diwel26   
13            felix   
14          Diwel26   
15         Пельмень   
16          Diwel26   
17             toby   
18          Diwel26   
19             Bach   
20          Diwel26   
21  sirop de fraise   
22          Diwel26   
23          Edu638™   
24          Diwel26   
25           vaccoc   
26          Diwel26   
27              Gui   
28          Diwel26   
29           Arthur   
30          Diwel26   
31             Simo   
32          Diwel26   
33         pancakes   
34          Diwel26   
35           LoKrox   
36          Diwel26   
37         dir egal   
38          Diwel26   
39             stef   
40          Diwel26   
41       mariomoron   
42         

In [93]:
df =  df.iloc[:2, :]
print(df)

   Player Name  \
0      Diwel26   
1  حسین شهبازی   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        Deck  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [94]:


# Assuming `df` is your DataFrame
# Combine every two rows into a single row
combined_rows = []
for i in range(0, len(df), 2):  # Iterate in steps of 2
    row1 = df.iloc[i]  # Winner
    row2 = df.iloc[i + 1] if i + 1 < len(df) else None  # Loser (Handle odd rows gracefully)
    
    if row2 is not None:
        combined_rows.append({
            'player_name': row1['Player Name'],
            'player_name2': row2['Player Name'],
            'deck': row1['Deck'],
            'deck2': row2['Deck'],
            'result': row1['Result'],  # Should be 1
            'result2': row2['Result'],  # Should be 0
        })

# Convert the combined rows to a DataFrame
combined_df1 = pd.DataFrame(combined_rows)

# Display the resulting DataFrame
print(combined_df1)

df = combined_df1


  player_name player_name2  \
0     Diwel26  حسین شهبازی   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        deck  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [95]:
df

,player_name,player_name2,deck,deck2,result,result2
0,Diwel26,حسین شهبازی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0,1


In [96]:
# Convert the deck column to the expected type
result_df['deck'] = result_df['deck'].apply(lambda x: [int(card) for card in x])

# Convert the result_df column types to match the schema
result_df = result_df.astype({
    'player_name': 'string',
    'deck': 'object',  # Ensure this is treated as an array in the Feature Store
    'result': 'string'
})

In [97]:
combined_rows = []
for i in range(0, len(result_df), 2):  # Iterate in steps of 2
    row1 = result_df.iloc[i]
    row2 = result_df.iloc[i + 1] if i + 1 < len(result_df) else None  # Handle odd-length DataFrame
    if row2 is not None:
        combined_rows.append({
            'player_name': row1['player_name'],
            'player_name2': row2['player_name'],
            'deck': row1['deck'],
            'deck2': row2['deck'],
            'result': row1['result'],  # Should be 1
            'result2': row2['result'],  # Should be 0
        })

# Convert the combined rows to a DataFrame
combined_df = pd.DataFrame(combined_rows)

# Display the resulting DataFrame
print(combined_df)

    player_name player_name2  \
0     #PVLPJP2Y   #PLYJVUQY2   
1    #8PRLRYYCV    #92VG2CPY   
2     #2G8LQRCG   #2PCUY9U80   
3    #Y9QL09VGV   #9GJJGYL8P   
4    #9RRYG9P9U   #80J0LUCP8   
..          ...          ...   
995   #8QYG0UYL    #JJP88C2V   
996  #P2UG2RCVU   #8VCY9828U   
997  #2JCP2VULL    #JC98JLVP   
998   #PP82G80Y     #UQURJ20   
999   #2RQGQ208   #92YYP9C9C   

                                                                                                                                                                                                                                                                                                                  deck  \
0    [0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0

In [98]:
result_df = combined_df

combined_df

,player_name,player_name2,deck,deck2,result,result2
0,#PVLPJP2Y,#PLYJVUQY2,"[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]",1,0
1,#8PRLRYYCV,#92VG2CPY,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
2,#2G8LQRCG,#2PCUY9U80,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
3,#Y9QL09VGV,#9GJJGYL8P,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...]",1,0
4,#9RRYG9P9U,#80J0LUCP8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...]",1,0
...,...,...,...,...,...,...
995,#8QYG0UYL,#JJP88C2V,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
996,#P2UG2RCVU,#8VCY9828U,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0

In [99]:
# Upload the dataframe one-hot encoded on Hopsworks

import os
import hopsworks
import pandas as pd


# 1. Read your Hopsworks API Key
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

# 3. Log in to your Hopsworks project
project = hopsworks.login()  # or hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()



# 4. Create (or get) a feature group
cr_feature_group = fs.get_or_create_feature_group(
    name="clash_royale_dataset_onehotencoding",
    version=4,
    description="Clash Royale cleaned match features with one-hot encoded decks",
    primary_key=["player_name"],  # Update this if your DataFrame's primary key is different
    online_enabled=True
)

# 5. Insert the DataFrame into the feature group
cr_feature_group.insert(result_df)

print("DataFrame inserted into Hopsworks Feature Store successfully!")

2025-01-05 21:10:32,392 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 21:10:32,402 INFO: Initializing external client
2025-01-05 21:10:32,402 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 21:10:34,456 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164438/fs/1155141/fg/1393592


Uploading Dataframe: 100.00% |██████████| Rows 1000/1000 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: clash_royale_dataset_onehotencoding_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164438/jobs/named/clash_royale_dataset_onehotencoding_4_offline_fg_materialization/executions
DataFrame inserted into Hopsworks Feature Store successfully!


In [100]:
# 1. Read your Hopsworks API Key from a file or environment variable
#    For example, if stored in 'hopsworks-api-key.txt'
with open('../data/hopsworks-api-key.txt', 'r') as f:
    api_key = f.read().strip()

# 2. Set the environment variable for Hopsworks
os.environ["HOPSWORKS_API_KEY"] = api_key

# 3. Log in to your Hopsworks project
project = hopsworks.login()  # or hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

# 4. Suppose we already have a DataFrame named df (e.g., from the previous step)
#    that has columns like: "player_name", "deck", "crowns", "result", "battle_time", etc.
#    Make sure df is defined before this script or above in the same script:
# df = ...

# 5. Create (or get) a feature group
cr_feature_group = fs.get_or_create_feature_group(
    name="clash_royale_onehotencoding",
    version=4,
    description="Clash Royale cleaned match features",
    primary_key=["player_name"],  # columns that uniquely identify each row
    online_enabled=True
)

# 6. Insert data into the feature group
cr_feature_group.insert(df)

print("DataFrame inserted into Hopsworks Feature Store successfully!")


2025-01-05 21:10:52,004 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-05 21:10:52,030 INFO: Initializing external client
2025-01-05 21:10:52,031 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-05 21:10:54,258 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164438
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164438/fs/1155141/fg/1394625


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: clash_royale_onehotencoding_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164438/jobs/named/clash_royale_onehotencoding_4_offline_fg_materialization/executions
DataFrame inserted into Hopsworks Feature Store successfully!


In [101]:
air_quality_roll_fg = fs.get_feature_group(
    name='clash_royale_onehotencoding',
    version=4,
)

In [103]:
df_aqi= air_quality_roll_fg.read()
df_aqi

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.26s) 


,player_name,player_name2,deck,deck2,result,result2
0,Diwel26,حسین شهبازی,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]",0,1


In [104]:
clash_royale_dataset_fg = fs.get_feature_group(
    name='clash_royale_dataset_onehotencoding',
    version=4,
)

In [105]:
df_aqi_dataset= clash_royale_dataset_fg.read()
df_aqi_dataset

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.82s) 


,player_name,player_name2,deck,deck2,result,result2
0,#PRG9UCG8C,#2J9JGLUJP,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
1,#QY8Q9LGY,#2R0PJUPVU,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
2,#URQJURR0,#YPG29JVR,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
3,#2V8G9RPQ8,#20GL9LPLC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
4,#PUVYRQRC,#2JC9PQ2L,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
...,...,...,...,...,...,...
620,#YYRPRU82V,#8R2YU8LPV,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1,0
621,#LRUU0L22,#2C9CJVYQ8,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,